# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [3]:
import pandas as pd
import numpy as np
import math
import json

from sklearn.model_selection import train_test_split

% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [4]:
portfolio.shape

(10, 6)

In [79]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [6]:
portfolio.describe()

,difficulty,duration,reward
count,10.000000,10.000000,10.000000
mean,7.700000,6.500000,4.200000
std,5.831905,2.321398,3.583915
min,0.000000,3.000000,0.000000
25%,5.000000,5.000000,2.000000
50%,8.500000,7.000000,4.000000
75%,10.000000,7.000000,5.000000
max,20.000000,10.000000,10.000000


In [7]:
profile.shape

(17000, 5)

In [8]:
profile.head(10)

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN
5,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0
6,118,20170925,None,8ec6ce2a7e7949b1bf142def7d0e0586,NaN
7,118,20171002,None,68617ca6246f4fbc85e91a2a49552598,NaN
8,65,20180209,M,389bc3fa690240e798340f5a15918d5c,53000.0
9,118,20161122,None,8974fc5686fe429db53ddde067b88302,NaN


In [9]:
profile.tail(10)

,age,became_member_on,gender,id,income
16990,70,20160310,F,79edb810789c447e8d212a324b44cc16,39000.0
16991,118,20160116,None,a9a20fa8b5504360beb4e7c8712f8306,NaN
16992,21,20170917,F,400d0536e8794cbb855b0d882d67cbda,72000.0
16993,60,20180505,M,cb23b66c56f64b109d673d5e56574529,113000.0
16994,118,20151211,None,c02b10e8752c4d8e9b73f918558531f7,NaN
16995,45,20180604,F,6d5f3a774f3d4714ab0c092238f3a1d7,54000.0
16996,61,20180713,M,2cb4f97358b841b9a9773a7aa05a9d77,72000.0
16997,49,20170126,M,01d26f638c274aa0b965d24cefe3183f,73000.0
16998,83,20160307,F,9dc1421481194dcd9400aec7c9ae6366,50000.0
16999,62,20170722,F,e4052622e5ba45a8b96b59aba68cf068,82000.0


In [10]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


In [11]:
transcript.shape

(306534, 4)

In [12]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [13]:
transcript[transcript['time']!= 0].head(5)

,event,person,time,value
15561,offer viewed,78afa995795e4d85b5d9ceeca43f5fef,6,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
15562,offer viewed,a03223e636434f42ac4c3df47e8bac43,6,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
15563,transaction,102e9454054946fda62242d2e176fdce,6,{'amount': 19.88}
15564,offer completed,102e9454054946fda62242d2e176fdce,6,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0...
15565,transaction,02c083884c7d45b39cc68e1314fec56c,6,{'amount': 1.44}


In [14]:
transcript.tail()

,event,person,time,value
306529,transaction,b3a1272bc9904337b331bf348c3e8c17,714,{'amount': 1.5899999999999999}
306530,transaction,68213b08d99a4ae1b0dcb72aebd9aa35,714,{'amount': 9.53}
306531,transaction,a00058cf10334a308c68e7631c529907,714,{'amount': 3.61}
306532,transaction,76ddbd6576844afe811f1a3c0fbb5bec,714,{'amount': 3.5300000000000002}
306533,transaction,c02b10e8752c4d8e9b73f918558531f7,714,{'amount': 4.05}


In [15]:
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


In [ ]:
#WORKSPACE

In [ ]:
portfolio
profile
transcript

In [6]:
df = transcript.copy()
df.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [ ]:
# Convert event from categorical to numerical values
# event_type = {'offer received': 0, 'offer viewed': 1, 'transaction': 2, 'offer completed': 3}
# transcript_labels_converted_df['event'] = transcript_labels_converted_df['event'].map(event_type).values
# transcript_labels_converted_df.head()

In [7]:
profile_id_as_index = profile.copy()
profile_id_as_index.set_index('id', inplace=True) 

In [8]:
def is_offer_used(person_id, offer_id, completed_offers):
    offer_used = completed_offers[(completed_offers['person'] == person_id) & (completed_offers['offer_id'] == offer_id)]
    return int(not offer_used.empty)

In [12]:

completed_offers = df[df['event'] == 'offer completed'].copy()
completed_offers['offer_id'] = completed_offers.apply(lambda row: row.value['offer_id'], axis=1)

# NOTE: Using 1000 data samples
viewed_offers_df = df[df['event'] == 'offer viewed'].iloc[0:1000].copy()
viewed_offers_df['offer_used_label'] = viewed_offers_df.apply(lambda row: is_offer_used(row['person'], row['value']['offer id'], completed_offers), axis=1)


valid_offers_df = pd.DataFrame(columns=['age', 'became_member_on', 'gender', 'income', 'event', 'person', 'time', 'value', 'offer_used_label'])

for idx, viewed_offer in viewed_offers_df.iterrows():
    valid_offers_df.loc[idx] = profile_id_as_index.loc[viewed_offer['person']].append(viewed_offer)
    

In [14]:
valid_offers_df.head()

,age,became_member_on,gender,income,event,person,time,value,offer_used_label
12650,65,20180209,M,53000.0,offer viewed,389bc3fa690240e798340f5a15918d5c,0,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},1
12651,53,20170916,O,52000.0,offer viewed,d1ede868e29245ea91818a903fec04c6,0,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0
12652,69,20160814,F,57000.0,offer viewed,102e9454054946fda62242d2e176fdce,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},1
12653,20,20160711,F,30000.0,offer viewed,02c083884c7d45b39cc68e1314fec56c,0,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0
12655,39,20140527,M,51000.0,offer viewed,be8a5d1981a2458d90b255ddc7e0d174,0,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0


In [22]:
# Split data into labels and features
valid_offers_used = valid_offers_df['offer_used_label'].copy()
valid_offers_features = valid_offers_df.drop(['offer_used_label'], axis=1)

In [23]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(valid_offers_features, 
                                                    valid_offers_used, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

In [29]:
# Check out the training and test sets
X_train.head()
y_train.head()

X_test.head()
y_test.head()

,age,became_member_on,gender,income,event,person,time,value
13593,20,20150917,M,66000.0,offer viewed,85bed66eecbf4340af74bbf0a363e1e0,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
13318,30,20180130,M,55000.0,offer viewed,88876be460544c19a852db9c55a04099,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
13094,53,20151030,M,74000.0,offer viewed,4438fece00ed4f1a8b108c3cd85b19c1,0,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
14027,42,20180410,O,89000.0,offer viewed,adf7a0dbdb1e438ead376fcde5c87177,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
13774,42,20180102,F,71000.0,offer viewed,f52b70a5d2044363bcef800c2f9efe73,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
